#### This project entailed
* Conceptualing and executing a visualization. The map is the first component of dashboard project. For this map, I found and combined multiple data sources to illustrate what I found interesting in the data. The map illustrates states with the highest proportion (relative to their populations) of admissions for substance abuse treatment. It also indicates the three most common substances that are abused, as indicated by these treatment admissions (excluding alcohol).
* Working with massive data sets. Data for 23 years' of case details for substance abuse treatment admissions were combined for this analysis. This totaled 41,000,000 records.
* Extensive data wrangling.
* Sluething the source of problematic data. When preparing the final map, some data were missing; I tracked down the source of the omission.


#### This notebook includes all the data wrangling for the project.
#### While other projects entailed different types of data wrangling, 
#### this project has widest ranging data wrangling, and the largest data sets. 

#### This notebook reflects the final python code, after missing data were identified and the data wrangling was changed appropriately.

In [1]:
import pandas as pd
import numpy as np
pd.set_option('mode.chained_assignment', None)
# 
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


import warnings
warnings.filterwarnings("ignore")

In [2]:
# Create data file with one row for each treatment admissions 1992 - 2014


years = range(1992, 2015)
pieces = []

for year in years: 
    path = 'C:/python/A%d.tsv' % year
    frame = pd.read_csv(path, sep='\t', usecols=['GENDER', 'RACE', 'STFIPS', 'SUB1','YEAR'], low_memory=False)
#    frame = frame_unf[(frame_unf.sub1 == 2) & (frame_unf.stfips == 25)] 
    pieces.append(frame)

df = pd.concat(pieces, ignore_index = True)

In [3]:
df.head()

,YEAR,GENDER,RACE,STFIPS,SUB1
0,1992,2,4,39,2
1,1992,1,5,26,2
2,1992,2,4,31,3
3,1992,1,5,31,2
4,1992,2,5,27,2


In [4]:
df.to_pickle ("C:/Users/rmbrm/Documents/DS_WP/Dashboard/initial_data", compression='infer', protocol=4)

In [5]:
raw = pd.read_pickle("C:/Users/rmbrm/Documents/DS_WP/Dashboard/initial_data")

In [6]:
# create a dataframe tallying yearly substance abuse admission by type and other variables for future use
# NOT PART OF THE MAP PROJECT; created and filed for future use.

years = range(1992, 2015)
pieces = []

for year in years: 
    year_string = str(year)
    dfcx = pd.DataFrame(raw.groupby('YEAR').get_group(year).reset_index()).drop('index', axis = 1)
    
    df = dfcx['GENDER'].value_counts().to_frame()
    df.index.names = ['year_string']
    df = df.transpose()
    df = df.rename({'GENDER': 'year_string'}, axis='index')
    gender = df.rename(index=str, columns={2: "female", 1: "male", -9: "gender_miss"})
    
    
    df = dfcx['SUB1'].value_counts().to_frame()
    df.index.names = ['year_string']
    df = df.transpose()
    df = df.rename({'SUB1': 'year_string'}, axis='index')
    sub1 = df.rename(index=str, columns={2: "ALCOHOL", 3: "COCAINE", 4: "MARIJ", 5: "HEROINE", 7: "OPIATES", 10: "METH"})
    
    df = dfcx['RACE'].value_counts().to_frame()
    df.index.names = ['year_string']
    df = df.transpose()
    df = df.rename({'RACE': 'year_string'}, axis='index')
    df = df.rename(index=str, columns={4: "black", 5: "white"})
    race = df.filter(['black', 'white'])
    
    df = dfcx['YEAR'].value_counts().to_frame()
    df.index.names = ['year_string']
    df = df.transpose()
    df = df.rename({'YEAR': 'year_string'}, axis='index')
    total = df.rename(index=str, columns={year: "total"})
    demo = pd.concat([sub1, gender, race, total], axis=1)
    pieces.append(demo)
tallies = pd.concat(pieces, ignore_index = True)

In [7]:
# add a variable for year
# not part of the map project; file was created for another part of this project.

when = pd.Series([1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014], index =[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22])


In [8]:
# not part of the map project; file was created for another part of this project.
tallies ['YEAR'] = when

In [9]:
# not part of the map project; file was created for another part of this project.
tallies.to_pickle ("C:/Users/rmbrm/Documents/DS_WP/Dashboard/annual_counts", compression='infer', protocol=4)

#### Data wrangling for map making: Selecting, formatting and combining data.
#### Step 1: Using most recent data, find the three most popular drugs in each state.

In [10]:
df= pd.read_pickle("C:/Users/rmbrm/Documents/DS_WP/Dashboard/initial_data")

In [11]:
dfa = df.rename(index=str, columns={"YEAR": "year", "GENDER": "gender", "RACE": "race", "STFIPS": "stfips", "SUB1": "sub1"})

In [12]:
dfa.tail()

,year,gender,race,stfips,sub1
41047260,2014,1,5,8,10
41047261,2014,1,4,6,5
41047262,2014,2,5,41,5
41047263,2014,1,13,6,10
41047264,2014,1,2,8,10


In [13]:
dfa.to_pickle ("C:/Users/rmbrm/Documents/DS_WP/Dashboard/map_data", compression='infer', protocol=4)

In [14]:
# select most current data for map
dfb = dfa.loc[dfa['year'] == 2014]


In [15]:
# keep variables of interest
dfb.drop(dfb.iloc[:, 0:3], inplace=True, axis=1)
dfb.head()

,stfips,sub1
39432907,29,3
39432908,26,2
39432909,18,2
39432910,17,2
39432911,18,2


In [16]:
# select most popular drugs
filter_list = [3, 4, 5, 7, 10]
dfc = dfb[dfb.sub1.isin(filter_list)]

In [17]:
# create variable for counts of drugs by state

dfd = dfc.groupby(['stfips', 'sub1'], as_index=False).size().reset_index().rename(columns={0: 'count'})

In [18]:
dfd.sort_values(['stfips', 'count'], ascending=[True, False], inplace=True)


In [19]:
dfd.head(10)

,stfips,sub1,count
1,1,4,2187
3,1,7,1438
4,1,10,1076
0,1,3,912
2,1,5,562
6,2,4,589
7,2,5,545
9,2,10,317
8,2,7,266
5,2,3,59


In [20]:
dfd = dfd.set_index('sub1').groupby('stfips')['count'].nlargest(3).reset_index()

In [21]:
dfd.tail(10)

,stfips,sub1,count
143,54,4,471
144,55,5,2191
145,55,4,2016
146,55,7,1102
147,56,4,948
148,56,10,747
149,56,7,320
150,72,5,600
151,72,4,500
152,72,3,127


In [22]:
dfd.to_pickle ("C:/Users/rmbrm/Documents/DS_WP/Dashboard/map_data_1", compression='infer', protocol=4)

In [23]:
dfd.sort_values(['stfips', 'count'], ascending=[True, False], inplace=True)

In [24]:
dfd.head()

,stfips,sub1,count
0,1,4,2187
1,1,7,1438
2,1,10,1076
3,2,4,589
4,2,5,545


In [25]:
# Replace state codes with state names
dfd.stfips = dfd.stfips.replace({1: "Alabama",
2: "Alaska",
4: "Arizona",
5: "Arkansas",
6: "California",
8: "Colorado",
9: "Connecticut",
10: "Delaware",
11: "DC",
12: "Florida",
13: "Georgia",
15: "Hawaii",
16: "Idaho",
17: "Illinois",
18: "Indiana",
19: "Iowa",
20: "Kansas",
21: "Kentucky",
22: "Louisiana",
23: "Maine",
24: "Maryland",
25: "Massachusetts",
26: "Michigan",
27: "Minnesota",
28: "Mississippi",
29: "Missouri",
30: "Montana",
31: "Nebraska",
32: "Nevada",
33: "New Hampshire",
34: "New Jersey",
35: "New Mexico",
36: "New York",
37: "North Carolina",
38: "North Dakota",
39: "Ohio",
40: "Oklahoma",
41: "Oregon",
42: "Pennsylvania",
44: "Rhode Island",
45: "South Carolina",
46: "South Dakota",
47: "Tennessee",
48: "Texas",
49: "Utah",
50: "Vermont",
51: "Virginia",
53: "Washington",
54: "West Virginia",
55: "Wisconsin",
56: "Wyoming",
72: "Puerto Rico"})

In [26]:
df = dfd

In [27]:
df['total'] = df['count'].groupby(df['stfips']).transform('sum')

In [28]:
df.sub1 = df.sub1.replace({4: "Marijuana: ",
                           5: "Heroin: ",
                           7: "Opiates: ",
                           3: "Cocaine: ",
                           10: "Meth: "})

In [29]:
df.head()

,stfips,sub1,count,total
0,Alabama,Marijuana:,2187,4701
1,Alabama,Opiates:,1438,4701
2,Alabama,Meth:,1076,4701
3,Alaska,Marijuana:,589,1451
4,Alaska,Heroin:,545,1451


In [30]:
df.to_pickle ("C:/Users/rmbrm/Documents/DS_WP/Dashboard/map_data_2", compression='infer', protocol=4)

In [31]:
df['total'] = df['count'].groupby(df['stfips']).transform('sum')

In [32]:
# count as a string
df['cnt_str'] = df['count'].astype(str)

In [33]:
# 
df1 = df.loc[[0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147, 150]]

In [34]:
# 
df2 = df.loc[[1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 31, 34, 37, 40, 43, 46, 49, 52, 55, 58, 61, 64, 67, 70, 73, 76, 79, 82, 85, 88, 91, 94, 97, 100, 103, 106, 109, 112, 115, 118, 121, 124, 127, 130, 133, 136, 139, 142, 145, 148, 151]]

In [35]:
# 
df3 = df.loc[[2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38, 41, 44, 47, 50, 53, 56, 59, 62, 65, 68, 71, 74, 77, 80, 83, 86, 89, 92, 95, 98, 101, 104, 107, 110, 113, 116, 119, 122, 125, 128, 131, 134, 137, 140, 143, 146, 149, 152]]

In [36]:
# 
df1['first'] = df1['sub1'] + df1['cnt_str']

In [37]:
# 
df2['second'] = df2['sub1'] + df2['cnt_str']

In [38]:
# 
df3['third'] = df3['sub1'] + df3['cnt_str']

In [39]:
# 
df3.drop(['total'], axis=1)

,stfips,sub1,count,cnt_str,third
2,Alabama,Meth:,1076,1076,Meth: 1076
5,Alaska,Meth:,317,317,Meth: 317
8,Arizona,Marijuana:,4033,4033,Marijuana: 4033
11,Arkansas,Opiates:,1312,1312,Opiates: 1312
14,California,Marijuana:,24379,24379,Marijuana: 24379
17,Colorado,Heroin:,6204,6204,Heroin: 6204
20,Connecticut,Cocaine:,4305,4305,Cocaine: 4305
23,Delaware,Opiates:,660,660,Opiates: 660
26,DC,Marijuana:,607,607,Marijuana: 607
29,Florida,Heroin:,3004,3004,Heroin: 3004


In [40]:
df2_merge = df2.drop(['sub1', 'count', 'cnt_str', 'total'], axis=1)

In [41]:
df3_merge = df3.drop(['sub1', 'count', 'cnt_str', 'total'], axis=1)

In [42]:
df1_merge = df1.drop(['sub1', 'count', 'cnt_str'], axis=1)

In [43]:
step1 = pd.merge(df1_merge, df2_merge, on="stfips")

In [44]:
final = pd.merge(step1, df3_merge, on="stfips")

In [45]:
final.to_pickle ("C:/Users/rmbrm/Documents/DS_WP/Dashboard/map_data_3", compression='infer', protocol=4)

In [46]:
final.head(15)

,stfips,total,first,second,third
0,Alabama,4701,Marijuana: 2187,Opiates: 1438,Meth: 1076
1,Alaska,1451,Marijuana: 589,Heroin: 545,Meth: 317
2,Arizona,12602,Meth: 4487,Heroin: 4082,Marijuana: 4033
3,Arkansas,4371,Meth: 1645,Marijuana: 1414,Opiates: 1312
4,California,110777,Meth: 48113,Heroin: 38285,Marijuana: 24379
5,Colorado,22148,Meth: 9113,Marijuana: 6831,Heroin: 6204
6,Connecticut,33927,Heroin: 21770,Marijuana: 7852,Cocaine: 4305
7,Delaware,5105,Heroin: 3388,Marijuana: 1057,Opiates: 660
8,DC,2798,Heroin: 1257,Cocaine: 934,Marijuana: 607
9,Florida,15439,Opiates: 7662,Marijuana: 4773,Heroin: 3004


#### Step 2: Using population data for the year with the most recent admissions data,
#### calculate percent of each state's population that was admitted for treatment.

In [47]:
# Remove alcohol from 2014 admits data
dfb = dfa.loc[dfa['sub1'] != 2]

In [48]:
admits_by_state = dfb['stfips'].value_counts().reset_index()

In [49]:
dfd = admits_by_state.rename(index=str, columns={"index": "stfips", "stfips": "admits_2014"})

In [50]:
dfe = pd.DataFrame(dfd)

In [51]:
dfe.sort_values(['stfips'], ascending=[True], inplace=True)

In [52]:
df_admits = dfe

In [53]:
dfe.to_pickle("C:/Users/rmbrm/Documents/DS_WP/Dashboard/admits", compression='infer', protocol=4)

In [54]:
dff = dfe.set_index(['stfips'])

In [55]:
# Read in 2014 population data file
df_pop = pd.read_csv("C:/Users/rmbrm/Documents/DS_WP/Dashboard/pop_2014.txt", sep='\t')

In [56]:
df_pop.set_index(['state'], inplace=True)
df_pop.head()

,pop_2014
state,
Alabama,4840037
Alaska,736759
Arizona,6706435
Arkansas,2964800
California,38701278


In [57]:
df_codes = pd.read_csv("C:/Users/rmbrm/Documents/DS_WP/Dashboard/state_codes.txt", sep='\t')

In [58]:
df_codes.set_index(['state'], inplace=True)
df_codes.head()

,stfips,postal
state,,
Alabama,1,AL
Alaska,2,AK
Arizona,4,AZ
Arkansas,5,AR
California,6,CA


In [59]:
df_pop_codes = pd.concat([df_pop, df_codes], axis=1)
df_pop_codes.head()

,pop_2014,stfips,postal
state,,,
Alabama,4840037,1,AL
Alaska,736759,2,AK
Arizona,6706435,4,AZ
Arkansas,2964800,5,AR
California,38701278,6,CA


In [60]:
df_pop_codes.to_pickle ("C:/Users/rmbrm/Documents/DS_WP/Dashboard/pop_and_codes", compression='infer', protocol=4)

In [61]:
df_p_c_w = df_pop_codes.reset_index()

In [62]:
df_p_c_f = df_p_c_w.set_index(['stfips'])
df_p_c_f.head()

,state,pop_2014,postal
stfips,,,
1,Alabama,4840037,AL
2,Alaska,736759,AK
4,Arizona,6706435,AZ
5,Arkansas,2964800,AR
6,California,38701278,CA


In [63]:
df_admits.set_index(['stfips'], inplace=True)

In [64]:
dfpct = pd.concat([df_p_c_f, df_admits], axis=1)

In [65]:
dfpct['percent'] = dfpct['admits_2014'] / dfpct['pop_2014']

In [66]:
dfpct_r = dfpct.round({'admits_2014': 0, 'percent': 2})

In [67]:
dfpct_r.to_pickle("C:/Users/rmbrm/Documents/DS_WP/Dashboard/percent_edited", compression='infer', protocol=4)

#### At this point, data for five states were showing up as NaN. Stepping through the data wrangling, I identified the problem: The 2015, which was on a different website from the other files, was apparently not finalized, as it did not contain data from those states. So I redid the data wrangling with the most recent, complete data, data from 2014.

In [68]:
substances = pd.read_pickle("C:/Users/rmbrm/Documents/DS_WP/Dashboard/map_data_3")

In [69]:
percent = pd.read_pickle("C:/Users/rmbrm/Documents/DS_WP/Dashboard/percent_edited")

In [70]:
# drop the state for which data is missing from the 2014 substance admissions data file, South Carolina
percent.drop([45], inplace=True)

In [71]:
percent.drop(['state'], axis=1, inplace=True)

In [72]:
percent.reset_index(inplace=True)

In [73]:
substances.rename(index=str, columns={"stfips": "state"}, inplace=True)

In [74]:
substances.reset_index(inplace=True)

In [75]:
percent.reset_index(inplace=True)

In [76]:
dfmap = pd.concat([substances, percent], axis=1)

In [77]:
dfpct.head()

,state,pop_2014,postal,admits_2014,percent
stfips,,,,,
1,Alabama,4840037,AL,223631,0.046204
2,Alaska,736759,AK,28540,0.038737
4,Arizona,6706435,AZ,258949,0.038612
5,Arkansas,2964800,AR,169623,0.057212
6,California,38701278,CA,3119945,0.080616


In [78]:
dfmap.to_pickle("C:/Users/rmbrm/Documents/DS_WP/Dashboard/map_final", compression='infer', protocol=4)

In [79]:
import pandas as pd

import plotly
plotly.tools.set_credentials_file(username="Ross.Brown.Ph.D.", api_key='yPNZCAkYOyi7wAKtZrSM')
import plotly.plotly as py
import plotly.offline as pyo
# from plotly.graph_objs import *
import plotly.plotly as py
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from plotly import tools
from plotly.graph_objs import Bar, Data, Figure, Layout, Marker, Scatter
init_notebook_mode(connected=True)
import cufflinks as cf
cf.set_config_file(offline=False, world_readable=True, theme='ggplot')

C:\Users\rmbrm\Anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:558: DeprecationWarning:

plotly.graph_objs.YAxis is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.YAxis
  - plotly.graph_objs.layout.scene.YAxis


C:\Users\rmbrm\Anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:531: DeprecationWarning:

plotly.graph_objs.XAxis is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.XAxis
  - plotly.graph_objs.layout.scene.XAxis




In [80]:
df = pd.read_pickle("C:/Users/rmbrm/Documents/DS_WP/Dashboard/map_final")

In [81]:
#for col in df.columns:
#    df[col] = df[col].astype(str)

scl = [[0.0, 'rgba(181, 189, 189, 0.8)'],[0.33, '#8BBBD9'],[0.66, '#2987D9'],\
            [1.0, '#012340']]

df['text'] = 'admissions as % of Pop' + '<br>' +\
    'Top Three:' + '<br>' +\
    df['first'] + '<br>' +\
    df['second'] + '<br>' +\
    df['third']
    
data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df['postal'],
        z = df['percent'].astype(float),
        locationmode = 'USA-states',
        text = df['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Substance Abuse <br>Admissions <br>(Percent of Population)")
        ) ]

layout = dict(
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
        annotations = [
            dict(
            x=0.5004254919715793,
            y=-0.16191064079952971,
            showarrow=False,
            text='Excluding alcohol. Data for South Carolina is unavailable',
            xref='paper',
            yref='paper'
            )
        ]
)



fig = go.Figure(data=data, layout=layout)
pyo.iplot(fig)